In [1]:
import numpy as np
import pandas as pd
from scipy import interpolate

In [2]:
def Contract_Calculation(Recom_Contract, UD_Measure,UD_Contract,UD_Pt_Cohort,Rebate_noVBC_flat, Rebate_VBC_flat,Rebate_noVBC_table,Rebate_VBC_table, MarketShare_table,Entresto_Utilizer_Perc,Script_PMPM):    
  
    import numpy as np
    import pandas as pd
    from scipy import interpolate

    # Pre-determined input
    
    CHF_Pt=150000*0.136
    Cost_per_Script_Gross=580
    
    Recom_Pt_cohort=UD_Pt_Cohort
    Performance_assumption=pd.read_csv('data/performance_assumption.csv')
    Performance_assumption.set_index(['Cohort','Measure'],inplace=True)
    Recom_Measure_all=pd.read_csv('data/recom_measure.csv')
    Recom_Measure_all.set_index(['Cohort','Measure'],inplace=True)
    Recom_Measure=Recom_Measure_all.loc[Recom_Pt_cohort]
    Pt_Info=pd.read_csv('data/patient_cohort_basic_info.csv')
    #Recom_Contract=pd.read_csv('C:\\Users\\wuyabo\\Recom_contract.csv')
    
    Recom_Performance_assumption=[]
    UD_Performance_assumption=[]
    Recom_Merge=[]
    UD_Merge=[]
    Recom_Rebate_Adj_Perc=[]
    UD_Rebate_Adj_Perc=[]

    #Calculate Total Script and Gross Revenue
    
    Entresto_Utilizer=CHF_Pt*Entresto_Utilizer_Perc
    Ultimate_MarketShare=MarketShare_table['MarketShare'].max()
    MarketShare_table['MM']=MarketShare_table['MarketShare']/Ultimate_MarketShare*Entresto_Utilizer
    Total_MM=MarketShare_table['MM'].sum()
    Total_Script=Total_MM*Script_PMPM
    Gross_Revenue=Cost_per_Script_Gross*Total_Script/1000000
    
    #Calculate Rebate
    
    if Rebate_noVBC_table.shape[0]==0:
        Rebate_noVBC=Rebate_noVBC_flat
    else:
        for i in range(4):
            if Ultimate_MarketShare>=Rebate_noVBC_table.iat[i,0] and Ultimate_MarketShare<=Rebate_noVBC_table.iat[i,1]:
                row_num=i
        Rebate_noVBC=Rebate_noVBC_table.iat[row_num,2]
    
    if Rebate_VBC_table.shape[0]==0:
        Rebate_VBC=Rebate_VBC_flat
    else:
        for i in range(4):
            if Ultimate_MarketShare>=Rebate_VBC_table.iat[i,0] and Ultimate_MarketShare<=Rebate_VBC_table.iat[i,1]:
                row_num=i
        Rebate_VBC=Rebate_VBC_table.iat[row_num,2]

    # Calculate financial results    
        
    Type = ['Recom','UD']
    Pt_cohort={'Recom':Recom_Pt_cohort , 'UD': UD_Pt_Cohort} 
    Contract_name={'Recom':Recom_Contract,'UD':UD_Contract}    
    Performance_assumption_name={'Recom':Recom_Performance_assumption,'UD':UD_Performance_assumption}
    Merge_name={'Recom':Recom_Merge,'UD':UD_Merge}
    Measure_name={'Recom':Recom_Measure,'UD':UD_Measure}
    Rebate_Adj_Perc={'Recom':Recom_Rebate_Adj_Perc,'UD':UD_Rebate_Adj_Perc}
    Rebate = {'noVBC': Rebate_noVBC, 'VBC': Rebate_VBC} 
    
    Type = ['Recom','UD']   
    Scenario_list=['Worst','Worse','Mid','Better','Best']
    VBC_list=['noVBC','VBC']
    
    for i in Type:
        Performance_assumption_name[i]=Performance_assumption.loc[Pt_cohort[i]]
        
        for scenario in Scenario_list:
            for VBC in VBC_list:
                Performance_assumption_name[i]['total_'+scenario+'_'+VBC]=Performance_assumption_name[i]['Medical_'+scenario]+Performance_assumption_name[i]['Rx_Before_Rebate_'+scenario]*(1-Rebate[VBC])
                Performance_assumption_name[i]['total_'+scenario+'_'+VBC].fillna(Performance_assumption_name[i]['Medical_'+scenario],inplace=True)
        
        Merge_name[i]=Measure_name[i].merge(Performance_assumption_name[i],left_on='Measure', right_on='Measure', suffixes=(False, False))
        
        Merge_name[i]['Worse_Diff']= Merge_name[i]['total_Worse_VBC'].astype(float)- Merge_name[i]['Target'].astype(float)
        Merge_name[i]['Worse_Perc']=np.where( Merge_name[i]['Scoring Method']==1,1+ Merge_name[i]['Worse_Diff']/ Merge_name[i]['Target'],1- Merge_name[i]['Worse_Diff']/ Merge_name[i]['Target'])
        Merge_name[i]['Mid_Diff']= Merge_name[i]['total_Mid_VBC'].astype(float)- Merge_name[i]['Target'].astype(float)
        Merge_name[i]['Mid_Perc']=np.where( Merge_name[i]['Scoring Method']==1,1+ Merge_name[i]['Mid_Diff']/ Merge_name[i]['Target'],1- Merge_name[i]['Mid_Diff']/ Merge_name[i]['Target'])
        Merge_name[i]['Better_Diff']= Merge_name[i]['total_Better_VBC'].astype(float)- Merge_name[i]['Target'].astype(float)
        Merge_name[i]['Better_Perc']=np.where( Merge_name[i]['Scoring Method']==1,1+ Merge_name[i]['Better_Diff']/ Merge_name[i]['Target'],1- Merge_name[i]['Better_Diff']/ Merge_name[i]['Target'])
            
        Worse_Performance=Merge_name[i]['Weight'].dot(Merge_name[i]['Worse_Perc'])
        Mid_Performance=Merge_name[i]['Weight'].dot(Merge_name[i]['Mid_Perc'])
        Better_Performance=Merge_name[i]['Weight'].dot(Merge_name[i]['Better_Perc'])
        
        x=[0,Contract_name[i].iloc[0][4],Contract_name[i].iloc[0][3],Contract_name[i].iloc[0][0],Contract_name[i].iloc[0][1],999]
        y=[Contract_name[i].iloc[0][5],Contract_name[i].iloc[0][5],0,0,Contract_name[i].iloc[0][2],Contract_name[i].iloc[0][2]]
        f = interpolate.interp1d(x, y)
        Rebate_Adj_Perc[i]=[f(Mid_Performance),Contract_name[i].iloc[0][5],Contract_name[i].iloc[0][2],f(Worse_Performance),f(Better_Performance)]
    
    #Produce output table - Pharma's net revenue  
    
    data = {'Scenario': ['Best Estimate', 'Worst', 'Best', 'Lower End', 'Higher End'], 
                'NoVBC Gross Revenue': [Gross_Revenue, 'NA', 'NA','NA','NA'], 
                'NoVBC Base Rebate Payout': [Gross_Revenue*Rebate_noVBC, 'NA', 'NA','NA','NA'], 
                'NoVBC Outcome Based Rebate Adjustment': [0, 'NA', 'NA','NA','NA'], 
                'NoVBC Net Rebate Payout': [Gross_Revenue*Rebate_noVBC, 'NA', 'NA','NA','NA'], 
                'NoVBC Net Revenue': [Gross_Revenue-Gross_Revenue*Rebate_noVBC, 'NA', 'NA','NA','NA']} 
    Output_Pharma_Net_Revenue = pd.DataFrame(data) 
    
    for i in Type:
        Output_Pharma_Net_Revenue[i+ 'VBC Gross Revenue']=Gross_Revenue
        Output_Pharma_Net_Revenue[i+'VBC Base Rebate Payout']=Gross_Revenue*Rebate_VBC
        Output_Pharma_Net_Revenue[i+'VBC Outcome Based Rebate Adjustment']=Output_Pharma_Net_Revenue[i+'VBC Gross Revenue']*Rebate_Adj_Perc[i]*(-1)
        Output_Pharma_Net_Revenue[i+'VBC Net Rebate Payout']=Output_Pharma_Net_Revenue[i+'VBC Base Rebate Payout']+Output_Pharma_Net_Revenue[i+'VBC Outcome Based Rebate Adjustment']
        Output_Pharma_Net_Revenue[i+'VBC Net Revenue']=Output_Pharma_Net_Revenue[i+'VBC Gross Revenue']-Output_Pharma_Net_Revenue[i+'VBC Net Rebate Payout']
    
    #Produce output table - Plan's medical cost
    
    Baseline=Pt_Info.loc[Pt_Info['Population'] == UD_Pt_Cohort]
    
    NoVBC_Cost=Merge_name['UD'][['total_Mid_noVBC', 'total_Worst_noVBC','total_Best_noVBC','total_Worse_noVBC','total_Better_noVBC']].iloc[0]
    
    RecomVBC_Cost=Merge_name['Recom'][['total_Mid_VBC', 'total_Worst_VBC','total_Best_VBC','total_Worse_VBC','total_Better_VBC']].iloc[0]
    UDVBC_Cost=Merge_name['UD'][['total_Mid_VBC', 'total_Worst_VBC','total_Best_VBC','total_Worse_VBC','total_Better_VBC']].iloc[0]
    
    data = {'Scenario': ['Best Estimate', 'Worst', 'Best', 'Lower End', 'Higher End'], 
                'Baseline Total Cost': [Baseline.iat[0,1]*Baseline.iat[0,2]/1000000, 'NA', 'NA','NA','NA'],
                'NoVBC Total Cost':[x * Baseline.iat[0,1]/1000000 for x in NoVBC_Cost],
                'NoVBC Rebate Adjustment':[0,0,0,0,0],
                'NoVBC Total Cost (After Rebate Adj)':[x * Baseline.iat[0,1]/1000000 for x in NoVBC_Cost],
                'RecomVBC Total Cost':[x * Baseline.iat[0,1]/1000000 for x in RecomVBC_Cost]}

    Output_Medical_Cost=pd.DataFrame(data) 

    Output_Medical_Cost['RecomVBC Rebate Adjustment']=-Output_Pharma_Net_Revenue['RecomVBC Outcome Based Rebate Adjustment']
    Output_Medical_Cost['RecomVBC Total Cost (After Rebate Adj)']=Output_Medical_Cost['RecomVBC Total Cost']+Output_Medical_Cost['RecomVBC Rebate Adjustment']

    Output_Medical_Cost['UDVBC Total Cost']=[x * Baseline.iat[0,1]/1000000 for x in UDVBC_Cost]
    Output_Medical_Cost['UDVBC Rebate Adjustment']=-Output_Pharma_Net_Revenue['UDVBC Outcome Based Rebate Adjustment']
    Output_Medical_Cost['UDVBC Total Cost (After Rebate Adj)']=Output_Medical_Cost['UDVBC Total Cost']+Output_Medical_Cost['UDVBC Rebate Adjustment']

    Output_Medical_Cost=Output_Medical_Cost.T
    header=Output_Medical_Cost.iloc[0]
    Output_Medical_Cost=Output_Medical_Cost[1:]
    Output_Medical_Cost.columns=header
    
    Output_Medical_Cost.insert(0, 'Contract Type',['Baseline']*1+ ['Contract w/o VBC Payout']*3+['Contract with VBC Payout (Recommended)']*3+['Contract with VBC Payout (User Defined)']*3)
    Output_Medical_Cost.insert(1, 'Item',['Total Cost']*1+ ['Total Cost','Rebate Adjustment','Total Cost (After Rebate Adj)']*3)
    
    Output_Medical_Cost.reset_index()  
    Output_Medical_Cost.set_index(['Contract Type'],inplace=True)
    
     #Transpose pharma net revenue table
    
    Output_Pharma_Net_Revenue=Output_Pharma_Net_Revenue.T
    header=Output_Pharma_Net_Revenue.iloc[0]
    Output_Pharma_Net_Revenue=Output_Pharma_Net_Revenue[1:]
    Output_Pharma_Net_Revenue.columns=header
    
    Output_Pharma_Net_Revenue.insert(0, 'Contract Type', ['Contract w/o VBC Payout']*5+['Contract with VBC Payout (Recommended)']*5+['Contract with VBC Payout (User Defined)']*5)
    Output_Pharma_Net_Revenue.insert(1, 'Item', ['Gross Revenue','Base Rebate Payout','Outcome Based Rebate Adjustment','Net Rebate Payout','Net Revenue']*3)
    
        
    #Produce output table - Pharma's rebate payout
    
    Output_Rebate = Output_Pharma_Net_Revenue[Output_Pharma_Net_Revenue['Item'].str.contains('Rebate')]
    Output_Pharma_Net_Revenue.reset_index()
    pos=[1,2,6,7,11,12]
    Output_Pharma_Net_Revenue.drop(Output_Pharma_Net_Revenue.index[pos], inplace=True)
    Output_Pharma_Net_Revenue.set_index(['Contract Type'],inplace=True)
    Output_Rebate.set_index(['Contract Type'],inplace=True)
 
    return Output_Pharma_Net_Revenue,Output_Rebate, Output_Medical_Cost
#    return  Output_Pharma_Net_Revenue, Output_Rebate,Output_Medical_Cost

In [3]:
data={'Marketshare_L':[0,0.05,0.1,0.15],
     'Marketshare_H':[0.05,0.1,0.15,0.20],
     'Rebate':[0.3,0.35,0.4,0.45]}

Rebate_noVBC_table=pd.DataFrame(data, columns = ['Marketshare_L', 'Marketshare_H', 'Rebate']) 

data={'Marketshare_L':[0,0.05,0.1,0.15],
     'Marketshare_H':[0.05,0.1,0.15,0.20],
     'Rebate':[0.3,0.35,0.35,0.45]}

Rebate_VBC_table=pd.DataFrame(data, columns = ['Marketshare_L', 'Marketshare_H', 'Rebate']) 

data={'Month':[1,2,3,4,5,6,7,8,9,10,11,12],
      'MarketShare':[0.02,0.03,0.04,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06]}

MarketShare_table=pd.DataFrame(data, columns = ['Month','MarketShare']) 

data = {'Measure': ['CHF Related Average Cost per Patient', 'CHF Related Hospitalization Rate', 'NT-proBNP Change %','LVEF LS Mean Change %'], 
                'Target': [33162, 978, -0.3,0.05], 
                'Weight': [0.5,0.25,0,0.25]} 
UD_Measure = pd.DataFrame(data, columns = ['Measure', 'Target', 'Weight']) 

data = {'Perf_Range_U_Min': [1], 
                'Perf_Range_U_Max': [1.2], 
                'Adj_Limit_U': [0.2],
                'Perf_Range_L_Min': [1],
                'Perf_Range_L_Max': [0.8],
                'Adj_Limit_L': [-0.2]} 
UD_Contract = pd.DataFrame(data, columns = ['Perf_Range_U_Min','Perf_Range_U_Max','Adj_Limit_U','Perf_Range_L_Min','Perf_Range_L_Max', 'Adj_Limit_L']) 
data = {'Perf_Range_U_Min': [1], 
                'Perf_Range_U_Max': [1.2], 
                'Adj_Limit_U': [0.15],
                'Perf_Range_L_Min': [1],
                'Perf_Range_L_Max': [0.8],
                'Adj_Limit_L': [-0.15]} 
Recom_Contract = pd.DataFrame(data, columns = ['Perf_Range_U_Min','Perf_Range_U_Max','Adj_Limit_U','Perf_Range_L_Min','Perf_Range_L_Max', 'Adj_Limit_L']) 
Rebate_noVBC_flat=0.1
Rebate_VBC_flat=0.1
Entresto_Utilizer_Perc=0.07
Script_PMPM=0.8
t1,t2,t3=Contract_Calculation(Recom_Contract, UD_Measure,UD_Contract,'CHF+AF (Recommended)', Rebate_noVBC_flat, Rebate_VBC_flat,Rebate_noVBC_table,Rebate_VBC_table, MarketShare_table,Entresto_Utilizer_Perc,Script_PMPM)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [4]:
t1

Scenario,Item,Best Estimate,Worst,Best,Lower End,Higher End
Contract Type,,,,,,
Contract w/o VBC Payout,Gross Revenue,6.84678,NA,NA,NA,NA
Contract w/o VBC Payout,Net Rebate Payout,2.39637,NA,NA,NA,NA
Contract w/o VBC Payout,Net Revenue,4.45041,NA,NA,NA,NA
Contract with VBC Payout (Recommended),Gross Revenue,6.84678,6.84678,6.84678,6.84678,6.84678
Contract with VBC Payout (Recommended),Net Rebate Payout,2.39323,3.42339,1.36936,2.75901,2.02588
Contract with VBC Payout (Recommended),Net Revenue,4.45355,3.42339,5.47743,4.08777,4.8209
Contract with VBC Payout (User Defined),Gross Revenue,6.84678,6.84678,6.84678,6.84678,6.84678
Contract with VBC Payout (User Defined),Net Rebate Payout,2.03176,3.76573,1.02702,2.44184,1.61992
Contract with VBC Payout (User Defined),Net Revenue,4.81503,3.08105,5.81977,4.40494,5.22686


In [5]:
t2

Scenario,Item,Best Estimate,Worst,Best,Lower End,Higher End
Contract Type,,,,,,
Contract w/o VBC Payout,Base Rebate Payout,2.39637,NA,NA,NA,NA
Contract w/o VBC Payout,Outcome Based Rebate Adjustment,0,NA,NA,NA,NA
Contract w/o VBC Payout,Net Rebate Payout,2.39637,NA,NA,NA,NA
Contract with VBC Payout (Recommended),Base Rebate Payout,2.39637,2.39637,2.39637,2.39637,2.39637
Contract with VBC Payout (Recommended),Outcome Based Rebate Adjustment,-0.00314393,1.02702,-1.02702,0.362635,-0.370495
Contract with VBC Payout (Recommended),Net Rebate Payout,2.39323,3.42339,1.36936,2.75901,2.02588
Contract with VBC Payout (User Defined),Base Rebate Payout,2.39637,2.39637,2.39637,2.39637,2.39637
Contract with VBC Payout (User Defined),Outcome Based Rebate Adjustment,-0.364617,1.36936,-1.36936,0.0454668,-0.77645
Contract with VBC Payout (User Defined),Net Rebate Payout,2.03176,3.76573,1.02702,2.44184,1.61992


In [15]:
t3

Scenario,Item,Best Estimate,Worst,Best,Lower End,Higher End
Contract Type,,,,,,
Baseline,Total Cost,25.1887,0,NA,NA,NA
Contract w/o VBC Payout,Total Cost,22.2225,0,19.428,23.2207,21.2243
Contract w/o VBC Payout,Rebate Adjustment,0,0,0,0,0
Contract w/o VBC Payout,Total Cost (After Rebate Adj),22.2225,25.017,19.428,23.2207,21.2243
Contract with VBC Payout (Recommended),Total Cost,22.2225,25.017,19.428,23.2207,21.2243
Contract with VBC Payout (Recommended),Rebate Adjustment,0.00314393,-1.02702,1.02702,-0.362635,0.370495
Contract with VBC Payout (Recommended),Total Cost (After Rebate Adj),22.2256,23.99,20.455,22.8581,21.5947
Contract with VBC Payout (User Defined),Total Cost,22.2225,25.017,19.428,23.2207,21.2243
Contract with VBC Payout (User Defined),Rebate Adjustment,0.364617,-1.36936,1.36936,-0.0454668,0.77645


In [14]:
t3.iloc[[0,1,2],[2]]=0

In [7]:
nl1 = 0
nl2 =5
nl3 =10
nl4 =15
nl5 =None

r1=5
r2=10
r3=15
r4=20
r5=None

p1=30
p2=35
p3=40
p4=45
p5=None


In [19]:
vbc_l = []
for i in range(5):
    
    vbc_l = vbc_l.extend(eval('l'+str(i+1)))

TypeError: 'int' object is not iterable

In [11]:
vbc_l = list(set([l1, l2, l3, l4, l5]))
vbc_r = list(set([r1, r2, r3, r4, r5]))
vbc_p = list(set([p1, p2, p3, p4, p5]))
input5 = {'Marketshare_L':vbc_l,
          'Marketshare_H':vbc_r,
          'Rebate':vbc_p}


In [12]:
input5

{'Marketshare_L': [0, 5, 10, 15, 20],
 'Marketshare_H': [5, 10, 15, 20, 25],
 'Rebate': [35, 40, 45, 50, 30]}

In [13]:
Rebate_VBC_table=pd.DataFrame(input5, columns = ['Marketshare_L', 'Marketshare_H', 'Rebate'])

In [16]:
Rebate_VBC_table

,Marketshare_L,Marketshare_H,Rebate
0,0.00,0.05,0.35
1,0.05,0.10,0.40
2,0.10,0.15,0.45
3,0.15,0.20,0.50
4,0.20,0.25,0.30


In [15]:

Rebate_VBC_table['Marketshare_L'] = Rebate_VBC_table['Marketshare_L']/100
Rebate_VBC_table['Marketshare_H'] = Rebate_VBC_table['Marketshare_H']/100
Rebate_VBC_table['Rebate'] = Rebate_VBC_table['Rebate']/100

In [8]:
novbc_l = list(filter(None,[nl1+1, nl2, nl3, nl4, nl5]))
#novbc_r = list(filter(None,[nr1, nr2, nr3, nr4, nr5]))
#novbc_p = list(filter(None,[np1, np2, np3, np4, np5]))


In [10]:

input4 = {'Marketshare_L':[novbc_l[i]-1 if i==0 else novbc_l[i] for i in range(len(novbc_l))],
         }

In [11]:
input4

{'Marketshare_L': [0, 5, 10, 15]}

In [16]:
data=[{'measures': 'Cost & Utilization Reduction', 'recom_value': None, 'tarrecom_value': None, 'taruser_value': None, 'probrecom': None, 'probuser': None, 'weight_recom': '60%', 'weight_user': '60%', 'highlight_recom': None, 'highlight_user': None, 'green_thres': None, 'yellow_thres': None, 'id': 0}, {'measures': 'CHF Related Average Cost per Patient', 'recom_value': '$30,020 ', 'tarrecom_value': '$29,630 ', 'taruser_value': '$29,630 ', 'probrecom': 'High', 'probuser': 'High', 'weight_recom': '30%', 'weight_user': '30%', 'highlight_recom': 'green', 'highlight_user': 'green', 'green_thres': 29630, 'yellow_thres': 28149, 'id': 1}, {'measures': 'CHF Related Hospitalization Rate', 'recom_value': '1150', 'tarrecom_value': '980', 'taruser_value': '980', 'probrecom': 'High', 'probuser': 'High', 'weight_recom': '30%', 'weight_user': '30%', 'highlight_recom': 'green', 'highlight_user': 'green', 'green_thres': 980, 'yellow_thres': 930, 'id': 2}, {'measures': 'Improving Disease Outcome', 'recom_value': None, 'tarrecom_value': None, 'taruser_value': None, 'probrecom': None, 'probuser': None, 'weight_recom': '40%', 'weight_user': '40%', 'highlight_recom': None, 'highlight_user': None, 'green_thres': None, 'yellow_thres': None, 'id': 9}, {'measures': 'LVEF LS Mean Change %', 'recom_value': '0%', 'tarrecom_value': '5%', 'taruser_value': '5%', 'probrecom': 'High', 'probuser': 'High', 'weight_recom': '40%', 'weight_user': '40%', 'highlight_recom': 'green', 'highlight_user': 'green', 'green_thres': 5, 'yellow_thres': 5.5, 'id': 11}]

In [17]:
row=data[1]

In [19]:
not('a'=='a')

False